
"""
Task 4 — Portfolio Optimization using MPT
- TSLA expected return from forecast (Task 3)
- BND/SPY expected returns from historical averages (annualized)
- Covariance from historical daily returns (TSLA/BND/SPY)
- Efficient Frontier (Monte Carlo), Max Sharpe & Min Volatility
- Outputs plot + CSV + summary report

Required inputs:
- CSV with historical OHLCV for TSLA, BND, SPY (must contain 'Date' and 'Adj Close')
- CSV with TSLA forecast from Task 3 (columns: Forecast, Lower, Upper; index as dates)
"""

In [1]:
#import  dependences
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# loaded function from the module

In [2]:
from src import portfolio_optimization
from src.portfolio_optimization import load_historical_prices,covariance_matrix_annual,compute_daily_returns,load_tsla_expected_return_from_forecast,expected_returns_annual,simulate_portfolios,pick_key_portfolios,plot_efficient_frontier,recommend_portfolio,write_summary

In [5]:
CONFIG = {
    # Paths to historical CSVs (from Task 1)
    "tsla_hist_csv": "F:/Time-Series-Forecasting-for-Portfolio-Management-Optimization-/data/scrap data/TSLA_data.csv",
    "bnd_hist_csv":  "F:/Time-Series-Forecasting-for-Portfolio-Management-Optimization-/data/scrap data/BND_data.csv",
    "spy_hist_csv":  "F:/Time-Series-Forecasting-for-Portfolio-Management-Optimization-/data/scrap data/SPY_data.csv",

    # Path to TSLA forecast results (from Task 3)
    "tsla_forecast_csv": "F:/Time-Series-Forecasting-for-Portfolio-Management-Optimization-/data/output/data/output/tesla_forecast_6months.csv",  # columns: Forecast, Lower, Upper

    # Columns
    "date_col": "Date",
    "price_col": "Close",  # fallback to 'Close' if not present

    # Output
    "output_dir": "data/output",
    "frontier_png": "efficient_frontier.png",
    "frontier_csv": "efficient_frontier_samples.csv",
    "summary_json": "portfolio_summary.json",
    "summary_txt": "portfolio_summary.txt",

    # Simulation
    "n_portfolios": 50000,
    "risk_free_rate": 0.02,  # annualized
    "trading_days": 252
}


## loading of historical data

In [3]:
#cfg=CONFIG, preference="max_sharpe"
preference="max_sharpe"
 # Load historical prices
prices = load_historical_prices()


In [6]:
df=pd.read_csv('F:/Time-Series-Forecasting-for-Portfolio-Management-Optimization-/data/output/arima_forecast (1).csv')  # columns: Forecast, Lower, Upper
print(df.head(10))
# print(forecasted_tsla.isna().sum())



         Date           0
0  2024-01-02  248.589829
1  2024-01-03  249.978745
2  2024-01-04  249.734335
3  2024-01-05  249.257801
4  2024-01-08  249.208264
5  2024-01-09  248.670971
6  2024-01-10  249.148629
7  2024-01-11  249.186808
8  2024-01-12  249.621606
9  2024-01-16  249.839300


# daily return

In [7]:
# -----------------------------
# 6) Main
# -----------------------------

    # Daily returns (for covariance)
daily_returns = compute_daily_returns(prices)
print(daily_returns)



                TSLA       BND       SPY
Date                                    
2015-07-02  0.040386  0.002472 -0.000916
2015-07-06 -0.001071  0.003452 -0.002846
2015-07-07 -0.042328  0.000984  0.006288
2015-07-08 -0.048231  0.002824 -0.016777
2015-07-09  0.011610 -0.003306  0.001809
...              ...       ...       ...
2025-07-24 -0.081970 -0.001232  0.000331
2025-07-25  0.035244  0.002468  0.004224
2025-07-28  0.030152 -0.001641 -0.000251
2025-07-29 -0.013483  0.005205 -0.002638
2025-07-30 -0.006725 -0.002589 -0.001259

[2534 rows x 3 columns]


# Expected Annual Return

In [8]:

    # TSLA expected annual return from forecast
tsla_exp_ann = load_tsla_expected_return_from_forecast()
print(tsla_exp_ann)


nan


f:\Time-Series-Forecasting-for-Portfolio-Management-Optimization-\src\portfolio_optimization.py:92: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_ret = forecast.pct_change().dropna()


 # Expected returns vecto

In [10]:

    # Expected returns vector
exp_ann = expected_returns_annual(daily_returns, tsla_exp_ann)
print(exp_ann)

TSLA         NaN
BND     0.019599
SPY     0.144844
dtype: float64


 # Annualized covariance matrix

In [11]:
    # Annualized covariance matrix
cov_ann = covariance_matrix_annual(daily_returns)
print(cov_ann.head(10))

          TSLA       BND       SPY
TSLA  0.350331  0.001861  0.052950
BND   0.001861  0.003016  0.001150
SPY   0.052950  0.001150  0.033272


# Frontier 

In [12]:

# Simulate frontier
frontier = simulate_portfolios(exp_ann, cov_ann)
os.makedirs("output_dir", exist_ok=True)
frontier.to_csv(os.path.join("output_dir", "frontier_csv"), index=False)


In [13]:

frontier.head()

,Volatility,Return,Sharpe,TSLA,BND,SPY
0,0.280972,NaN,NaN,0.379321,0.123785,0.496895
1,0.268444,NaN,NaN,0.399724,0.309355,0.290921
2,0.386781,NaN,NaN,0.596652,0.071224,0.332123
3,0.343516,NaN,NaN,0.571696,0.393181,0.035123
4,0.405418,NaN,NaN,0.638705,0.085700,0.275595


# Key portfolios

In [ ]:

# Key portfolios
max_sharpe, min_vol = pick_key_portfolios(frontier)


In [ ]:

# Plot
png_path = plot_efficient_frontier(frontier, max_sharpe, min_vol, cfg)


In [ ]:

    # Recommendation (default: max Sharpe)
recommended = recommend_portfolio(max_sharpe, min_vol, preference=preference)


In [ ]:

    # Write summaries
json_path, txt_path = write_summary(exp_ann, cov_ann, max_sharpe, min_vol, recommended, cfg)

print("\n=== Outputs ===")
print("Efficient frontier plot:", png_path)
print("Frontier samples CSV:", os.path.join(cfg["output_dir"], cfg["frontier_csv"]))
print("Summary (JSON):", json_path)
print("Summary (TXT):", txt_path)
print("\nDone.")

